# Agentic Email Text Vector Store
* Read a specific inbox from and to date
* Store the text as embbedings in vector store

In [1]:
'''
    WARNING CONTROL to display or ignore all warnings
'''
import warnings; warnings.simplefilter('ignore')     #switch betweeb 'default' and 'ignore'
import traceback

''' Set debug flag to view extended error messages; else set it to False to turn off debugging mode '''
debug = True


In [6]:
import os
import sys
import litellm
os.environ['LITELLM_LOG'] = 'DEBUG' 
# litellm.set_verbose=True
os.environ['CHROMA_TELEMETRY_ENABLED']='false'
# os.environ["OLLAMA_BASE_URL"] = "http://192.168.2.200:5050"
litellm.api_base = "http://192.168.2.200:5050"

proj_dir = os.path.abspath(os.pardir)
sys.path.insert(1,proj_dir.split('wrangler/')[0])
from wrangler.modules.shipping.emailPulse import crew as cr
from wrangler.modules.shipping.emailPulse import agents as ag
from wrangler.modules.shipping.emailPulse import tasks as ta
from wrangler.modules.shipping.emailPulse.tools import read_tool as rt
# from wrangler.modules.shipping.emailPulse.tools import api_tools as at

''' restart initiate classes '''
if debug:
    import importlib
    cr = importlib.reload(cr)
    ag = importlib.reload(ag)
    ta = importlib.reload(ta)
    rt = importlib.reload(rt)

__desc__ = "crewai agents and tasks"
clsCrew = cr.agentWorkLoads(desc=__desc__)
print("\n%s class initialization and load complete!" % __desc__)

All functional CREW-libraries in EMAILPULSE-package of SHIPPING-module imported successfully!
model Class initialization complete
__propAttr__ Class initialization complete

crewai agents and tasks class initialization and load complete!


## Run Crew

In [7]:
inputs = {
    # "email_addr": 'devin@sahanafoundation.org',
    "from_date":'2025-01-01',
    "to_date" : '2025-03-31',
}
crew_output, vectstore_ids, vectorstore = clsCrew._run(inputs=inputs)

print(f"Stored IDS : {vectstore_ids}")
print(f"Vectorstore: {vectorstore}")
print(f"Tasks Output ...:\n{crew_output.tasks_output}")
print(f"Token Usage: {crew_output.token_usage}")

Overriding of current TracerProvider is not allowed


# Agent: Meticulous email content reader.
## Task: Execute the following steps in order: 1. Use the tool to read the email content 2. Filter the email from 2025-01-01 to 2025-03-31



# Agent: Meticulous email content reader.
## Thought: To accomplish the task of retrieving and organizing email content from 2025-01-01 to 2025-03-31, I should first use the Email Reader tool to fetch emails based on the specified date range. This will allow me to gather all relevant emails that fall within the given timeframe.
## Using tool: Email Reader
## Tool Input: 
"{\"tool_input\": \"filter emails from 2025-01-01 to 2025-03-31\"}"
## Tool Output: 
[{'subject': 'Re: Fwd: Enquiry from Botswana', 'from': '"Dominic König" <dominic@nursix.org>', 'body': "Oh great, thank you - I totally forgot that there must be an account already, \r\nsince I've posted before (EdenMobile, that was about). The passwort reset \r\nemail hasn't arrived yet, though.\r\n\r\nYes, it will help a lot if you can start a page. \r\

In [27]:
''' DEPRECATED only for testing '''
''' VECTORDB '''
_db_type = 'chromadb'
_db_root = "/home/nuwan/workloads/advantis/wrangler/data/shipping/emailPulse/def_job123"
_db_name = 'email'
from rezaware.modules.etl.loader import vectorDB as db
import importlib
db = importlib.reload(db)
clsVDB = db.dataWorkLoads(
    db_type=_db_type, 
    db_root=_db_root,
    db_name=_db_name
)
clsVDB.dbType, clsVDB.dbRoot, clsVDB.dbName

kwargs = {"SPLITTER" : "LANGCHAIN"}
documents = clsVDB.text_to_documents(
    text=crew_output.raw, 
    chunk_size=200, 
    overlap=10,
    **kwargs,
)
print(len(documents))

All functional VECTORDB-libraries in LOADER-package of ETL-module imported successfully!
__propAttr__ Class initialization complete
8


In [28]:
collection_name="nuwan"
vectorstore_ = clsVDB.store_vectors(
    documents=documents, # list of document documents
    collection=collection_name,   # the documents collection name
    embedding_fn=None, # embediing function to use
    )        
vectorstore_

## Verify vectorstore

In [4]:
print(len(vect_store.get()['documents']))
query="discussion about WordPress?"
retriever=vect_store.as_retriever()
docs = retriever.invoke(query)
print("\n".join([doc.page_content for doc in docs]))


NameError: name 'vect_store' is not defined

## Test tool function : Fetch Emails

In [21]:
clsEmail = rt.searchWorkLoads()
emails = clsEmail.fetch_emails(from_date='2025-01-01', to_date='2025-03-31')
emails

[{'subject': 'Re: Fwd: Enquiry from Botswana',
  'from': '"Dominic König" <dominic@nursix.org>',
  'body': "Oh great, thank you - I totally forgot that there must be an account already, \r\nsince I've posted before (EdenMobile, that was about). The passwort reset \r\nemail hasn't arrived yet, though.\r\n\r\nYes, it will help a lot if you can start a page. \r\nIs there a template I can follow?\r\n\r\nDominic\r\n\r\nonsdag 12 mars 2025 20:33:02 CET skrev  Devin Balkind:\r\n> Dominic, you have an existing WordPress user with username nursix and email\r\n> dominic@nursix.org. I upgraded you to WordPress admin access to admin and\r\n> sent a password reset email.\r\n> \r\n> Login url: https://sahanafoundation.org/login45871/\r\n> \r\n> If you want I can also start a page for you.\r\n> \r\n> Let me know.\r\n\r\n"},
 {'subject': 'Re: Fwd: Enquiry from Botswana',
  'from': 'Devin Balkind <devin@sahanafoundation.org>',
  'body': 'Dominic, you have an existing WordPress user with username nursix

## Alternative test - Get Labels

In [21]:
clsEmail = rt.searchWorkLoads()
labels=clsEmail.get_label_ids()
for label in labels:
    print(f"Label Name: {label['name']}, Label ID: {label['id']}")
 

Label Name: CHAT, Label ID: CHAT
Label Name: SENT, Label ID: SENT
Label Name: INBOX, Label ID: INBOX
Label Name: IMPORTANT, Label ID: IMPORTANT
Label Name: TRASH, Label ID: TRASH
Label Name: DRAFT, Label ID: DRAFT
Label Name: SPAM, Label ID: SPAM
Label Name: CATEGORY_FORUMS, Label ID: CATEGORY_FORUMS
Label Name: CATEGORY_UPDATES, Label ID: CATEGORY_UPDATES
Label Name: CATEGORY_PERSONAL, Label ID: CATEGORY_PERSONAL
Label Name: CATEGORY_PROMOTIONS, Label ID: CATEGORY_PROMOTIONS
Label Name: CATEGORY_SOCIAL, Label ID: CATEGORY_SOCIAL
Label Name: STARRED, Label ID: STARRED
Label Name: UNREAD, Label ID: UNREAD
Label Name: Proposal, Label ID: Label_1557621474489886905
Label Name: Finances, Label ID: Label_1650221986040434965
Label Name: Pictographs, Label ID: Label_169801410101587110
Label Name: Literature, Label ID: Label_6689679009541273485


In [5]:
from google_auth_oauthlib.flow import InstalledAppFlow

SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]  # Adjust the scope if needed
credentials_path = "/home/nuwan/workspace/advantis/wrangler/modules/shipping/emailPulse/tools/credentials.json"

# Run OAuth flow to get credentials
flow = InstalledAppFlow.from_client_secrets_file(credentials_path, SCOPES)
creds = flow.run_local_server(port=0)  # Opens a browser for authentication

# Print and save the refresh token
print("Refresh Token:", creds.refresh_token)
print("Access Token:", creds.token)

# Save the credentials to a file
with open("token.json", "w") as token_file:
    token_file.write(creds.to_json())


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=356050208895-fjal482l0s9iu6vtqrnh3epemav9jd9b.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A43455%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=6hTy5chHtlCQ4d4tiy9mm53PUAn1ch&access_type=offline
Refresh Token: 1//067OHEJtHgmakCgYIARAAGAYSNwF-L9Irc6SghYGSkpfxPYjKLuquykgJhoqnBkYBQr2gSRRo4MXNMAg21VuXxa7_nbXdW88dM_g
Access Token: ya29.a0AeXRPp4vWPxK6e_b2MFuGbywK-dZLkUsBR9a_hr5G-mwYw1h33kdAfIx0XwDj_FTzw671qTe0gUSOfTiTcjeuKotvSBM9jvR9X7F4tsgJgK-Z5TpOg9CZHkldbs36LnxIeWLji8KmMdJaoV2mr3hFl6ILjV4ZDV__7RU4uj6aCgYKAa8SARISFQHGX2Miuou_LrRmNKtNluF69ou49w0175
